# Trabalho prático AC

## Trabalho realizado por:

Margarida Vila-chã, 20210504

Miguel Duarte Silva, 20210504

Sebastião Lessa, 202103238


## Algoritmo escolhido e breve explicação:

O algoritmo escolhido pelo nosso grupo foi o Random Forrest.

Este algoritmo é um algoritmo de aprendizagem computacional supervisionado. Ele usa uma combinação de árvores de decisão aleatórias. O processo começa com a criação de um grande número de árvores (floresta), em que cada uma das árvores criadas é treinada com uma amostra diferente. Isto permite a criação de árvores suficientemente diferentes, o que leva a que elas juntas consigam lidar com uma ampla variedade de situações, reduzindo o risco de overffitting.

